# MNEflow basic example

# 1. Importing data

### 1.1.from MNE epochs

If you use MNE-python, all you need is to provide your epochs file (or list of epoch files) to mneflow.produce_tfrecords


In [1]:
%pylab inline
#get epochs using your mne-python pipeline
import os
from time import time
import mne
from mne.datasets import multimodal
os.chdir('/m/nbe/work/zubarei1/mneflow/')
import mneflow

mne.set_log_level(verbose='CRITICAL')

fname_raw = os.path.join(multimodal.data_path(), 'multimodal_raw.fif')
raw = mne.io.read_raw_fif(fname_raw)

cond = raw.acqparser.get_condition(raw, None)
#get the list of condition names
condition_names = [k for c in cond for k,v in c['event_id'].items()]

epochs_list = [mne.Epochs(raw, **c) for c in cond]

#here we concatenate epochs because each input file contains just one condition
#otherwise mneflow.produce_tfrecords can handle a list of epochs objects
epochs = mne.concatenate_epochs(epochs_list)
#pick only planar gradiometers
epochs = epochs.pick_types(meg='grad')
#print(epochs.info)

Populating the interactive namespace from numpy and matplotlib


### Convert epochs to TFRecord format

In [2]:
#Specify import options

import_opt = dict(savepath='../tfr/', #path where TFR files will be saved
                   out_name='mne_sample_epochs1', #name of TFRecords files
                   fs=600,
                   input_type='trials',
                   target_type='int',
                   picks = {'meg':'grad'},
                   scale=True, #apply baseline_scaling
                   crop_baseline=True, #remove baseline interval after scaling
                   decimate = 2,
                   scale_interval=(0,60), #indices in time axis corresponding to baseline interval
                   val_size=0.15, #validation set size set to 15% of all data
                   overwrite=False) 


#write TFRecord files and metadata file to disk
#meta = mneflow.produce_tfrecords([epochs],**import_opt)  
meta = mneflow.produce_tfrecords([epochs],**import_opt)  

Metadata file found, restoring


## Other import options
### 1.2 Saved mne.epochs (*-epo.fif) files
Alternatively, if your epochs are saved to disk provide a str (or list of str) with path(s) to your -epo.fif files

e.g. this will work

```python
epochs.save('test_saved_epochs.fif')
meta = mneflow.produce_tfrecords('test_saved_epochs.fif',**opt)
```
### 1.3. Arrays in *.mat or *.npz format
if the first argument is str mneflow.produce_tfrecords can also accept *.mat or *.npz format

e.g.

```python
data_path = '/m/nbe/scratch/braindata/izbrv/detection_data/'
filenames = [data_path +'sub' + str(i) + '-grad.npz' for i in range(1,4)]
meta = mneflow.produce_tfrecords(filenames,**opt)
```
In this case, specify iput_type='array', and also provide array_keys keyword argument

e.g. 

```python
array_keys={'X':'my_data_samples','y':'my_labels'}
```

### 1.4. Tuple of (data, labels)
Finally, if you have a more complex preprocessing pipeline, you can feed you data and labels as a tuple of arrays

```python
X = epochs.get_data()
y = epochs.events[:,2]
meta = mneflow.produce_tfrecords((X,y),**opt)
```


# 2. Initialize the dataset object using the generated metadata file

The dataset object includes several methods that allow experimenting with the dataset without the need to repeat the preprocessing or overwriting the TFRecord files each time.

For example, you can train the model using any subset of classes, channels or reduce the sampling rate by decimating across the time domain.

In [3]:
dataset = mneflow.Dataset(meta, train_batch = 200, class_subset=None, pick_channels=None, decim=None)

W1024 16:32:25.760249 140697714599680 deprecation_wrapper.py:119] From /m/nbe/work/zubarei1/mneflow/mneflow/data.py:137: The name tf.parse_single_example is deprecated. Please use tf.io.parse_single_example instead.

W1024 16:32:25.786967 140697714599680 deprecation.py:323] From /m/nbe/work/zubarei1/mneflow/mneflow/data.py:111: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


# 3. Choose from already implemented models

MNEflow pipeline consists of three major parts:
1. dataset
2. optimizer
3. computational graph

Each part has its own set of hyper-parameters and methods that can be tuned. See help for mneflow.Dataset
and mneflow.Optimizer and mneflow.models.Model for more details.
In this example will we use LF-CNN network


In [4]:
#specify optimizer parmeters
optimizer_params = dict(l1_lambda=3e-4,learn_rate=3e-4)

optimizer = mneflow.Optimizer(**optimizer_params)

In [5]:
#specify parameters specific to LF-CNN
lf_params = dict(n_ls=64, #number of latent factors
              filter_length=17, #convolutional filter length in time samples
              pooling = 5, #pooling factor
              stride = 5, #stride parameter for pooling layer
              padding = 'SAME',
              dropout = .5,
              model_path = import_opt['savepath']) #path for storing the saved model

#initialize the model using the dataset and optimizer objects, and the hyper-parameter dictionary
model = mneflow.models.LFCNN(dataset, optimizer, lf_params)

#this will initialize the iterators over the dataset,the computational graph and the optimizer
model.build()

W1024 16:32:25.829552 140697714599680 deprecation_wrapper.py:119] From /m/nbe/work/zubarei1/mneflow/mneflow/models.py:44: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1024 16:32:25.834677 140697714599680 deprecation_wrapper.py:119] From /m/nbe/work/zubarei1/mneflow/mneflow/models.py:45: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1024 16:32:25.835964 140697714599680 deprecation.py:323] From /m/nbe/work/zubarei1/mneflow/mneflow/models.py:68: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
W1024 16:32:25.893560 140697714599680 deprecation_wrapper.py:119] From /m/nbe/work/zu

X0: (?, 204, 151)
de-mix init : OK
dmx (?, 204, 1, 64)
lf-inp 

W1024 16:32:25.943397 140697714599680 deprecation.py:506] From /m/nbe/work/zubarei1/mneflow/mneflow/layers.py:63: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1024 16:32:25.954068 140697714599680 deprecation_wrapper.py:119] From /m/nbe/work/zubarei1/mneflow/mneflow/models.py:85: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W1024 16:32:25.963893 140697714599680 deprecation_wrapper.py:119] From /m/nbe/work/zubarei1/mneflow/mneflow/optimize.py:76: The name tf.losses.softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.softmax_cross_entropy instead.

W1024 16:32:25.987668 140697714599680 deprecation.py:323] From /u/62/zubarei1/unix/.conda/envs/py3ml/lib/python3.7/site-packages/tensorflow/python/ops/losses/losses_impl.py:121: add_dispatch_support.

(?, 204, 1, 64)
conv init : OK
lf-inp (?, 204, 1, 64)
f: (17, 1, 64, 1)
lf-out (?, 41, 1, 64)
fc ::: 2624 8
fc init : OK
X: (?, 204, 151, 1)
y_pred: (?, 8)
L1 penalty applied to weights
Initialization complete!


In [6]:
#train the model
start = time()
model.train(n_iter=3000,eval_step=250,min_delta=1e-6,early_stopping=1)
stop = time() - start
print('Trained in {:.2f}s'.format(stop))

i 0, tr_loss 2.63277, tr_acc 0.145 v_loss 2.5999, v_acc 0.148936
i 250, tr_loss 1.47392, tr_acc 0.715 v_loss 1.69491, v_acc 0.553191
i 500, tr_loss 0.779655, tr_acc 0.955 v_loss 1.01218, v_acc 0.808511
i 750, tr_loss 0.454264, tr_acc 1 v_loss 0.860577, v_acc 0.808511
i 1000, tr_loss 0.480628, tr_acc 1 v_loss 0.750533, v_acc 0.861702
i 1250, tr_loss 0.442505, tr_acc 1 v_loss 0.74687, v_acc 0.904255
i 1500, tr_loss 0.413581, tr_acc 1 v_loss 0.65524, v_acc 0.914894
i 1750, tr_loss 0.390058, tr_acc 1 v_loss 0.635465, v_acc 0.925532
i 2000, tr_loss 0.358462, tr_acc 1 v_loss 0.597493, v_acc 0.925532
i 2250, tr_loss 0.346232, tr_acc 1 v_loss 0.579263, v_acc 0.914894
i 2500, tr_loss 0.324752, tr_acc 1 v_loss 0.526663, v_acc 0.946809
i 2750, tr_loss 0.303648, tr_acc 1 v_loss 0.523416, v_acc 0.925532
i 3000, tr_loss 0.284246, tr_acc 1 v_loss 0.497532, v_acc 0.925532
Trained in 126.17s


# 4. Explore the trained model parameters
LFCNN allows to interpret the trained parameters in terms of toporaphies and the spectral properties of the latent sources contributing to each class. 

In [7]:
model.compute_patterns('patterns')
f = model.plot_patterns(sensor_layout='Vectorview-grad', sorting='best', spectra=False, scale=True)


ValueError: shapes (6946,204) and (151,64) not aligned: 204 (dim 1) != 151 (dim 0)

In [ ]:
#Plotting the confusion matrix allows to identify analyze whether cetrain classes 
#are systematically harder to classify 
f2 = model.plot_cm(dataset='validation', class_names=condition_names)

In [ ]:
print(meta['y_shape'])